In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD
from keras.constraints import maxnorm

Using TensorFlow backend.


In [5]:
#ModelCreate#Applies to All grid searches used.
def create_model(input_dim=13, optimizer='adam', init_mode='uniform', activation='relu', dropout_rate=-1, weight_constraint=-1, neurons=16):
	# create model
	model = Sequential()
	model.add(Dense(neurons, input_dim=input_dim, kernel_initializer=init_mode, activation=activation))        
	model.add(Dense(1, kernel_initializer=init_mode, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
	return model

In [6]:
def gsBatchEpoch(input_dim=13):
    model = KerasClassifier(build_fn=create_model, input_dim=input_dim, verbose=0)
    # define the grid search parameters
    batch_size = [10, 20, 40, 60, 80, 100]
    epochs = [10, 50, 100]
    param_grid = dict(batch_size=batch_size, epochs=epochs)
    return model, batch_size, epochs, param_grid

In [7]:
def gsOptimizer(input_dim=13, batch_size=10, epochs=100):
    #####GS2
    # create model
    model = KerasClassifier(build_fn=create_model, input_dim=input_dim, verbose=0, batch_size=batch_size, epochs=epochs)

    # define the grid search parameters
    optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
    param_grid = dict(optimizer=optimizer)
    
    return model, optimizer, param_grid

In [8]:
def gsInitMode(input_dim=13, batch_size=10, epochs=100, optimizer='adam'):
    # create model
    model = KerasClassifier(build_fn=create_model, input_dim=input_dim, optimizer=optimizer, verbose=0, batch_size=batch_size, epochs=epochs)
    init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
    param_grid = dict(init_mode=init_mode)
    return model, init_mode, param_grid

In [9]:
def gsActivation(input_dim=13, batch_size=10, epochs=100, optimizer='adam', init_mode='uniform'):
    # create model
    model = KerasClassifier(build_fn=create_model, input_dim=input_dim, optimizer=optimizer, init_mode=init_mode, verbose=0, batch_size=batch_size, epochs=epochs)
    activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
    param_grid = dict(activation=activation)
    
    return model, activation, param_grid

In [10]:
def gsNeurons(input_dim=13, batch_size=10, epochs=100, optimizer='adam', init_mode='uniform', activation='relu', dropout_rate=0.0, weight_constraint=0):
    # create model
    model = KerasClassifier(build_fn=create_model, input_dim=input_dim, optimizer=optimizer, init_mode=init_mode, activation=activation, dropout_rate=dropout_rate, weight_constraint=weight_constraint, verbose=0, batch_size=batch_size, epochs=epochs)
    neurons = [1, 5, 10, 15, 20, 25, 30]
    param_grid = dict(neurons=neurons)
    return model, neurons, param_grid

In [12]:
def studentSetup():
    ####Education Data Setup
    # load dataset
    dataset = pd.read_csv("slim-xAPI-Edu-Data.csv")
    #students failed if they are in class L.
    dataset['Failed'] = np.where(dataset['Class'] == 'L', True, False)
    dataset['gender'] = np.where(dataset['gender']=='M',1,0)
    dataset['Relation'] = np.where(dataset['Relation']=='Father',1,0)
    dataset['ParentAnsweringSurvey'] = np.where(dataset['ParentAnsweringSurvey'] == 'Yes', 1, 0)
    dataset['ParentschoolSatisfaction'] = np.where(dataset['ParentschoolSatisfaction'] == 'Yes', 1, 0)
    dataset['AbsentMoreThanWeek'] = np.where(dataset['StudentAbsenceDays'] == 'Above-7', 1, 0)
    dataset['Semester'] = np.where(dataset['Semester'] == 'F', 1, 0)
    X = dataset[['raisedhands', 'VisITedResources', 'SectionID', 'Topic', 'StageID', 'AnnouncementsView', 'Semester', 'Discussion', 'gender', 'Relation', 'ParentAnsweringSurvey', 'ParentschoolSatisfaction', 'AbsentMoreThanWeek']]
    Y = dataset[['Failed']]
    
    return X, Y

In [13]:
def colourSetup():
    dataset = pd.read_csv("colour-data.csv")
    X = dataset[['R', 'G', 'B']]
    Y = dataset[['Label']]
    return X, Y

In [14]:
def weatherSetup():
    dataset = pd.read_csv("modified-monthly-data-labelled.csv")
    X = dataset.iloc[:,0:61]
    Y = dataset.iloc[:,61]
    return X, Y

In [15]:
def printResult(grid_result):
        #Printing Results / Best Parameters.
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [ ]:
#####start of Student Data Analysis.
X, Y = studentSetup()

In [ ]:
##### Grid Search 1 - Batch Size and Epochs.
model, batch_size, epochs, param_grid = gsBatchEpoch(input_dim=X.shape[1])

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(np.array(X), np.array(Y))

In [ ]:
bestBatchSize = grid_result.best_params_.get('batch_size')
bestEpochs = grid_result.best_params_.get('epochs')
printResult(grid_result)

In [ ]:
##### Grid Search 2 - for best Optimizer.
model, optimizer, param_grid = gsOptimizer(input_dim=X.shape[1], batch_size=bestBatchSize, epochs=bestEpochs)

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(np.array(X), np.array(Y))

In [ ]:
#From GS2, we get the best optimizer.  Save this for future reference.  Pray it is not SGD.
bestOptimizer = grid_result.best_params_.get('optimizer')
printResult(grid_result)

In [ ]:
##### Grid Search 4 - for best init_mode
model, init_mode, param_grid = gsInitMode(input_dim=X.shape[1], batch_size=bestBatchSize, epochs=bestEpochs, optimizer=bestOptimizer)

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(np.array(X), np.array(Y))

In [ ]:
bestInitMode = grid_result.best_params_.get('init_mode')
printResult(grid_result)

In [ ]:
##### Grid Search 5 - for best Activation
model, activation, param_grid = gsActivation(input_dim=X.shape[1], batch_size=bestBatchSize, epochs=bestEpochs, optimizer=bestOptimizer, init_mode=bestInitMode)

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(np.array(X), np.array(Y))

In [ ]:
bestActivation = grid_result.best_params_.get('activation')
printResult(grid_result)

In [ ]:
##### Grid Search 7 - for best Neurons
model, neurons, param_grid = gsNeurons(input_dim=X.shape[1], batch_size=bestBatchSize, epochs=bestEpochs, optimizer=bestOptimizer, init_mode=bestInitMode, dropout_rate = 0, weight_constraint=0)

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(np.array(X), np.array(Y))

In [ ]:
bestNeurons = grid_result.best_params_.get('neurons')
printResult(grid_result)

In [ ]:
#####start of Color Data Analysis
X, Y = studentSetup()

In [ ]:
##### Grid Search 1 - Batch Size and Epochs.
model, batch_size, epochs, param_grid = gsBatchEpoch(input_dim=X.shape[1])
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)


In [ ]:
grid_result = grid.fit(np.array(X), np.array(Y))


In [ ]:
bestBatchSize = grid_result.best_params_.get('batch_size')
bestEpochs = grid_result.best_params_.get('epochs')
printResult(grid_result)

In [ ]:
##### Grid Search 2 - for best Optimizer.
model, optimizer, param_grid = gsOptimizer(input_dim=X.shape[1], batch_size=bestBatchSize, epochs=bestEpochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)

In [ ]:
grid_result = grid.fit(np.array(X), np.array(Y))

In [ ]:
#From GS2, we get the best optimizer.  Save this for future reference.  Pray it is not SGD.
bestOptimizer = grid_result.best_params_.get('optimizer')
printResult(grid_result)

In [ ]:
##### Grid Search 4 - for best init_mode
model, init_mode, param_grid = gsInitMode(input_dim=X.shape[1], batch_size=bestBatchSize, epochs=bestEpochs, optimizer=bestOptimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)

In [ ]:
grid_result = grid.fit(np.array(X), np.array(Y))

In [ ]:
bestInitMode = grid_result.best_params_.get('init_mode')
printResult(grid_resul

In [ ]:
bestInitMode = grid_result.best_params_.get('init_mode')
printResult(grid_result)

In [ ]:
##### Grid Search 5 - for best Activation
model, activation, param_grid = gsActivation(input_dim=X.shape[1], batch_size=bestBatchSize, epochs=bestEpochs, optimizer=bestOptimizer, init_mode=bestInitMode)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)

In [ ]:
grid_result = grid.fit(np.array(X), np.array(Y))

In [ ]:
bestActivation = grid_result.best_params_.get('activation')
printResult(grid_result)

In [ ]:
##### Grid Search 7 - for best Neurons
model, neurons, param_grid = gsNeurons(input_dim=X.shape[1], batch_size=bestBatchSize, epochs=bestEpochs, optimizer=bestOptimizer, init_mode=bestInitMode, dropout_rate = 0, weight_constraint=0)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)

In [ ]:
grid_result = grid.fit(np.array(X), np.array(Y))

In [ ]:
bestNeurons = grid_result.best_params_.get('neurons')
printResult(grid_result)

In [ ]:
#####start of Color Data Analysis
X, Y = colourSetup()

In [ ]:
##### Grid Search 1 - Batch Size and Epochs.
model, batch_size, epochs, param_grid = gsBatchEpoch(input_dim=X.shape[1])
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)

In [ ]:
grid_result = grid.fit(np.array(X), np.array(Y))

In [ ]:
bestBatchSize = grid_result.best_params_.get('batch_size')
bestEpochs = grid_result.best_params_.get('epochs')
printResult(grid_result)

In [ ]:
##### Grid Search 2 - for best Optimizer.
model, optimizer, param_grid = gsOptimizer(input_dim=X.shape[1], batch_size=bestBatchSize, epochs=bestEpochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)

In [ ]:
grid_result = grid.fit(np.array(X), np.array(Y))

In [ ]:
#From GS2, we get the best optimizer.  Save this for future reference.  Pray it is not SGD.
bestOptimizer = grid_result.best_params_.get('optimizer')
printResult(grid_result)

In [ ]:
##### Grid Search 4 - for best init_mode
model, init_mode, param_grid = gsInitMode(input_dim=X.shape[1], batch_size=bestBatchSize, epochs=bestEpochs, optimizer=bestOptimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)

In [ ]:
grid_result = grid.fit(np.array(X), np.array(Y))

In [ ]:
bestInitMode = grid_result.best_params_.get('init_mode')
printResult(grid_result)

In [ ]:
##### Grid Search 5 - for best Activation
model, activation, param_grid = gsActivation(input_dim=X.shape[1], batch_size=bestBatchSize, epochs=bestEpochs, optimizer=bestOptimizer, init_mode=bestInitMode)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)

In [ ]:
grid_result = grid.fit(np.array(X), np.array(Y))

In [ ]:
bestActivation = grid_result.best_params_.get('activation')
printResult(grid_result)

In [ ]:
##### Grid Search 7 - for best Neurons
model, neurons, param_grid = gsNeurons(input_dim=X.shape[1], batch_size=bestBatchSize, epochs=bestEpochs, optimizer=bestOptimizer, init_mode=bestInitMode, dropout_rate = 0, weight_constraint=0)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)

In [ ]:
grid_result = grid.fit(np.array(X), np.array(Y))

In [ ]:
bestNeurons = grid_result.best_params_.get('neurons')
printResult(grid_result)